# Jass Agent with Reinforcement Learning
### Fact sheet by Ruben Nunez & Jordan Suter

This fact sheet is highly inspired by the paper of Jacop Chapman and Mathias Lechner. <<Deep Q-Learning for Atari Breakout>>

We tried to implement the Jass-Agent similarly as described in the paper.


<figure>
    <img src="img/reinforcement.png" width="600"/>
</figure>

### table of contents:
* Setup
* Parameter
* Deep Q-Network
* Agent
* Environment
* Training (Jass loop)

### source directory
- https://de.mathworks.com/discovery/reinforcement-learning.html
- https://keras.io/examples/rl/deep_q_network_breakout/
- https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python/

UNO BOT mit sequential Model
- https://github.com/PhilippThoelke/uno-bot/blob/master/agent.py
- https://www.tensorflow.org/guide/keras/save_and_serialize

## Setup

In [21]:
import sys
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf

from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

from tensorflow import keras
from tensorflow.keras import layers, models

sys.path.append("../deep-jass-project")
# noinspection PyUnresolvedReferences
from agent_gen1 import AgentGen1
# noinspection PyUnresolvedReferences
from agent_gen2 import AgentGen2
# noinspection PyUnresolvedReferences
from agent_gen3 import AgentGen3
# noinspection PyUnresolvedReferences
from agent_gen4 import AgentGen4
# noinspection PyUnresolvedReferences
from agent_gen5 import AgentGen5
# noinspection PyUnresolvedReferences
from agent_helper import get_remaining_cards

## Parameter

In [22]:
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# In the Deepmind paper they use RMSProp however then Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0) # improves training time

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
trick_count = 0

epsilon_random_frames = 50000  # Number of frames to take random action and observe output
epsilon_greedy_frames = 1000000.0  # Number of frames for exploration

# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000 # Maximum replay length
update_after_actions = 1  # Train the model after 1 actions
update_target_network = 10000  # How often to update the target network
loss_function = keras.losses.Huber()  # Using huber loss for stability

## Deep Q-Network
This network learns an approximation of the Q-table, which is a mapping between the states and actions that an agent will take.
In every state are 36 actions, that can eventually be taken. The environment provides the state, and the action is chosen by selecting the larger of the 36 Q-values predicted in the output layer.

In [38]:
num_actions = 36

def create_UNO_model(input_size, output_size):
    # define the model architecture
    _model = models.Sequential()
    _model.add(layers.Dense(units=64, activation='relu', input_shape=(input_size,)))
    _model.add(layers.Dense(units=64, activation='relu'))
    _model.add(layers.Dense(units=64, activation='relu'))
    _model.add(layers.Dense(units=output_size, activation='linear'))

    # compile the model
    _model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return _model


def create_q_model():
    # inputs = layers.Input(shape=(84, 84, 4,)) # Network defined by the Deepmind paper [0,0,0,0,0,0,...,0,0,0,0,.......,0,0,0,..1,0,0,0,0,0]
    inputs = layers.Input(shape=(80,)) # Network defined by the Deepmind paper [0,0,0,0,0,0,...,0,0,0,0,.......,0,0,0,..1,0,0,0,0,0]

    # Convolutions on the frames on the screen
    #layer1 = layers.Conv2D(16, 2, strides=4, activation="relu")(inputs)
    layer1 = layers.Conv1D(16, 1, strides=4, activation="relu")(inputs)
    #layer2 = layers.Conv2D(64, 1, strides=2, activation="relu")(layer1)
    layer2 = layers.Conv1D(16, strides=4, activation="relu")(layer1)
    #layer3 = layers.Conv2D(64, 1, strides=1, activation="relu")(layer2)
    layer3 = layers.Conv1D(16, strides=4, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_UNO_model(151, 36)
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_UNO_model(151, 36)

## Agent

In [39]:
# Reinforcement Agent Implementation
# by Ruben Nunez & Jordan Suter

class AgentReinforcement(Agent):

    @staticmethod
    def calculate_trump_selection_score(cards, trump: int) -> int:
        trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
        no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]

        score = 0
        for card in cards:
            suit = int(card / 9)
            exact_card = card % 9
            score += trump_score[exact_card] if trump == suit else no_trump_score[exact_card]

        return score

    def __init__(self):
        super().__init__()
        self._rule = RuleSchieber() # we need a rule object to determine the valid cards

    def action_trump(self, obs: GameObservation) -> int:
        card_list = convert_one_hot_encoded_cards_to_int_encoded_list(obs.hand)
        threshold = 68
        scores = [0, 0, 0, 0]
        for suit in range(0, 4):
            trump_score = self.calculate_trump_selection_score(card_list, suit)
            scores[suit] = trump_score

        best_score = max(scores)
        best_suit = scores.index(best_score)

        if best_score <= threshold and obs.player < 1:
            return PUSH
        else:
            return best_suit

    def action_play_card(self, obs: GameObservation, _action) -> int:
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        if valid_cards[_action] == 1:
            card = _action
        else:
            card = np.random.choice(np.flatnonzero(valid_cards))

        return card

## Environment

In [72]:
class Environment:

    game = None

    agent_reinforcement = None
    agent_1 = None
    agent_2 = None
    agent_3 = None

    def __init__(self):
        self.agent_reinforcement = AgentReinforcement()
        self.agent_1 = AgentGen1()
        self.agent_2 = AgentGen1()
        self.agent_3 = AgentGen1()

    def new_game(self):
        rule = RuleSchieber()
        game = GameSim(rule=rule)

        # deal cards
        game.init_from_cards(hands=deal_random_hand(), dealer=SOUTH)
        obs = game.get_observation()

        trump = self.agent_reinforcement.action_trump(obs) # set trump
        game.action_trump(trump)

        if trump == PUSH: # set PUSH
            obs = game.get_observation()
            trump = self.agent_2.action_trump(obs) # set trump
            game.action_trump(trump) # tell the simulation the selected trump

        self.game = game

    def step(self, _action):
        obs = self.game.get_observation()

        if obs.player == 0:
            self.game.action_play_card(self.agent_reinforcement.action_play_card(obs, _action))
        if obs.player == 1:
            self.game.action_play_card(self.agent_1.action_play_card(obs))
        if obs.player == 2:
            self.game.action_play_card(self.agent_2.action_play_card(obs))
        if obs.player == 3:
            self.game.action_play_card(self.agent_3.action_play_card(obs))

        _done = self.game.is_done()
        _state = self.state(),
        _reward = 0

        if obs.player == 0:
            _reward = obs.points[0] - obs.points[1]

        return _reward, _done


    def state(self):
        obs = self.game.get_observation()
        if obs.current_trick is None:
            obs.current_trick = np.array([-1, -1, -1, -1])

        remaining_cards = get_remaining_cards(obs)
        valid_cards = self.game.rule.get_valid_cards_from_obs(obs)

        result = np.concatenate((obs.hand, valid_cards, remaining_cards, obs.tricks.reshape(36), obs.current_trick, np.array([obs.declared_trump]), np.array([obs.nr_cards_in_trick]),  np.array([obs.nr_played_cards])))

        return result


## Training (Jass loop)

In [ ]:
env = Environment()

# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
move_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()


while True:  # Run until solved
    env.new_game() # Start Game (80,)
    state = env.state()
    episode_reward = 0

    for time_step in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        move_count += 1

        # Use epsilon-greedy for exploration
        if move_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        reward, done = env.step(action)
        state_next = env.state()
        # state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if move_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability

            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if move_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, move_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    print("difference: " + str(env.game.get_observation().points[0] - env.game.get_observation().points[1]) + " @Episode: " + str(episode_count) + " running: reward = "  + str(running_reward), flush=True)

    if running_reward > 400:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

difference: -79 @Episode: 1 running: reward = -44.0
difference: -41 @Episode: 2 running: reward = -90.5
difference: -135 @Episode: 3 running: reward = -230.66666666666666
difference: -87 @Episode: 4 running: reward = -267.25
difference: -77 @Episode: 5 running: reward = -291.6
difference: -1 @Episode: 6 running: reward = -216.5
difference: 37 @Episode: 7 running: reward = -199.42857142857142
difference: 17 @Episode: 8 running: reward = -221.75
difference: -77 @Episode: 9 running: reward = -226.77777777777777
difference: 17 @Episode: 10 running: reward = -221.6
difference: -25 @Episode: 11 running: reward = -214.72727272727272
difference: 55 @Episode: 12 running: reward = -191.08333333333334
difference: -129 @Episode: 13 running: reward = -215.07692307692307
difference: -103 @Episode: 14 running: reward = -229.42857142857142
difference: -95 @Episode: 15 running: reward = -222.33333333333334
difference: -11 @Episode: 16 running: reward = -205.5625
difference: -21 @Episode: 17 running: re